# **Imports**

In [26]:
import torch

def format_pytorch_version(version):
  return version.split('+')[0]

TORCH_version = torch.__version__
TORCH = format_pytorch_version(TORCH_version)

def format_cuda_version(version):
  return 'cu' + version.replace('.', '')

CUDA_version = torch.version.cuda
CUDA = format_cuda_version(CUDA_version)

!pip install torch-scatter -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-sparse -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-cluster -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-spline-conv -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-geometric

Looking in links: https://pytorch-geometric.com/whl/torch-2.0.1+cu118.html
Looking in links: https://pytorch-geometric.com/whl/torch-2.0.1+cu118.html
Looking in links: https://pytorch-geometric.com/whl/torch-2.0.1+cu118.html
Looking in links: https://pytorch-geometric.com/whl/torch-2.0.1+cu118.html


In [27]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from torch_geometric.utils.convert import from_networkx
from torch_geometric.nn.models import Node2Vec
from torch_geometric.utils import train_test_split_edges
from torch_geometric.nn import GCNConv
from torch_geometric.utils import negative_sampling
from sklearn.metrics import roc_auc_score
import torch.nn.functional as F
from torch_geometric.transforms import RandomLinkSplit
import requests

# **Data acquisition**

In [28]:
URL = "https://snap.stanford.edu/data/facebook_combined.txt.gz"
response = requests.get(URL)
open("facebook_combined.txt.gz", "wb").write(response.content)

218576

# **Data cleansing and preparation**

In [29]:
all_edges = pd.read_csv('facebook_combined.txt.gz', compression='gzip', header = None, sep=' ')
all_edges.columns = ['source', 'target']
m = all_edges['source'] > all_edges['target']
all_edges.loc[m, ['source', 'target']] = (all_edges.loc[m, ['target', 'source']].values)
all_edges = all_edges.drop_duplicates()

G = nx.from_pandas_edgelist(all_edges, "source", "target", create_using = nx.Graph())
G.remove_edges_from(nx.selfloop_edges(G))
data = from_networkx(G)

num_features = 128
n2v_model = Node2Vec(edge_index = data.edge_index, embedding_dim = num_features, walk_length = 80, context_size = 10, walks_per_node = 10, sparse = True)
data.x = n2v_model.forward()

data = train_test_split_edges(data)
print(data)

/usr/local/lib/python3.10/dist-packages/torch_geometric/deprecation.py:22: UserWarning: 'train_test_split_edges' is deprecated, use 'transforms.RandomLinkSplit' instead
  warnings.warn(out)


Data(num_nodes=4039, x=[4039, 128], val_pos_edge_index=[2, 4411], test_pos_edge_index=[2, 8823], train_pos_edge_index=[2, 150000], train_neg_adj_mask=[4039, 4039], val_neg_edge_index=[2, 4411], test_neg_edge_index=[2, 8823])


# **Baseline model**

In [30]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GCNConv(num_features, 32)
        self.conv2 = GCNConv(32, 16)

    def encode(self):
        x = self.conv1(data.x, data.train_pos_edge_index) # convolution 1
        x = x.relu()
        return self.conv2(x, data.train_pos_edge_index) # convolution 2

    def decode(self, z, pos_edge_index, neg_edge_index): # only pos and neg edges
        edge_index = torch.cat([pos_edge_index, neg_edge_index], dim=-1) # concatenate pos and neg edges
        logits = (z[edge_index[0]] * z[edge_index[1]]).sum(dim=-1)  # dot product
        return logits

    def decode_all(self, z):
        prob_adj = z @ z.t() # get adj NxN
        return (prob_adj > 0).nonzero(as_tuple=False).t() # get predicted edge_list

In [31]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model, data = Net().to(device), data.to(device)
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.01)

In [32]:
def get_link_labels(pos_edge_index, neg_edge_index):
    # returns a tensor:
    # [1,1,1,1,...,0,0,0,0,0,..] with the number of ones is equel to the lenght of pos_edge_index
    # and the number of zeros is equal to the length of neg_edge_index
    E = pos_edge_index.size(1) + neg_edge_index.size(1)
    link_labels = torch.zeros(E, dtype=torch.float, device=device)
    link_labels[:pos_edge_index.size(1)] = 1.
    return link_labels


def train():
    model.train()

    neg_edge_index = negative_sampling(
        edge_index=data.train_pos_edge_index, #positive edges
        num_nodes=data.num_nodes, # number of nodes
        num_neg_samples=data.train_pos_edge_index.size(1)) # number of neg_sample equal to number of pos_edges

    optimizer.zero_grad()

    z = model.encode() #encode
    link_logits = model.decode(z, data.train_pos_edge_index, neg_edge_index) # decode

    link_labels = get_link_labels(data.train_pos_edge_index, neg_edge_index)
    loss = F.binary_cross_entropy_with_logits(link_logits, link_labels)
    loss.backward()
    optimizer.step()

    return loss


@torch.no_grad()
def test():
    model.eval()
    perfs = []
    for prefix in ["val", "test"]:
        pos_edge_index = data[f'{prefix}_pos_edge_index']
        neg_edge_index = data[f'{prefix}_neg_edge_index']

        z = model.encode() # encode train
        link_logits = model.decode(z, pos_edge_index, neg_edge_index) # decode test or val
        link_probs = link_logits.sigmoid() # apply sigmoid

        link_labels = get_link_labels(pos_edge_index, neg_edge_index) # get link

        perfs.append(roc_auc_score(link_labels.cpu(), link_probs.cpu())) #compute roc_auc score
    return perfs

# **Train model**

In [33]:
best_val_perf = test_perf = 0
for epoch in range(1, 101):
    train_loss = train()
    val_perf, tmp_test_perf = test()
    if val_perf > best_val_perf:
        best_val_perf = val_perf
        test_perf = tmp_test_perf
    log = 'Epoch: {:03d}, Loss: {:.4f}, Val: {:.4f}, Test: {:.4f}'
    if epoch % 10 == 0:
        print(log.format(epoch, train_loss, best_val_perf, test_perf))

Epoch: 010, Loss: 0.4688, Val: 0.9366, Test: 0.9333
Epoch: 020, Loss: 0.4358, Val: 0.9640, Test: 0.9613
Epoch: 030, Loss: 0.4203, Val: 0.9707, Test: 0.9710
Epoch: 040, Loss: 0.4100, Val: 0.9767, Test: 0.9765
Epoch: 050, Loss: 0.4044, Val: 0.9798, Test: 0.9796
Epoch: 060, Loss: 0.3991, Val: 0.9822, Test: 0.9814
Epoch: 070, Loss: 0.3964, Val: 0.9837, Test: 0.9825
Epoch: 080, Loss: 0.3950, Val: 0.9850, Test: 0.9834
Epoch: 090, Loss: 0.3933, Val: 0.9858, Test: 0.9843
Epoch: 100, Loss: 0.3899, Val: 0.9862, Test: 0.9846


# **Evaluate**

In [34]:
from torch_geometric.utils import to_networkx
from torch_geometric.data import Data

z = model.encode()
final_edge_index = model.decode_all(z)

data_final = Data(edge_index = final_edge_index, num_nodes=data.num_nodes)
G = to_networkx(data_final)
G.remove_edges_from(nx.selfloop_edges(G))
pred_edges = nx.to_pandas_edgelist(G)
result = pd.merge(all_edges, pred_edges, how="outer", indicator="Exists")
result = result.loc[result.Exists == "right_only"].drop(columns = ["Exists"])

m = result['source'] > result['target']
result.loc[m, ['source', 'target']] = (result.loc[m, ['target', 'source']].values)
result = result.drop_duplicates()
result

,source,target
88234,0,348
88235,0,349
88236,0,353
88237,0,354
88238,0,355
...,...,...
6386213,4014,4038
6386219,4020,4038
6386222,4023,4038
6386225,4027,4038


In [35]:
complete_graph_edges = (data_final.num_nodes * (data_final.num_nodes - 1)) / 2
print(result.shape[0] / complete_graph_edges)

0.38911683399877445
